In [ ]:
###################### Install all necessary libraries #########################

from google.colab import files
!pip install praw -q
!pip install yfinance --upgrade --no-cache-dir -q
!pip install yahoo_fin -q
!pip install yahoo_fin --upgrade -q
!pip install requests_html -q
!pip install mpld3 -q
!pip install fastapi -q
!pip install colabcode -q

################################################################################

     |████████████████████████████████| 163kB 8.0MB/s 
     |████████████████████████████████| 61kB 4.3MB/s 
     |████████████████████████████████| 6.3MB 12.2MB/s 
     |████████████████████████████████| 81kB 4.8MB/s 
     |████████████████████████████████| 92kB 4.9MB/s 
     |████████████████████████████████| 81kB 4.7MB/s 
     |████████████████████████████████| 81kB 6.2MB/s 
ERROR: pyppeteer 0.2.5 has requirement urllib3<2.0.0,>=1.25.8, but you'll have urllib3 1.24.3 which is incompatible.
     |████████████████████████████████| 890kB 9.4MB/s 
     |████████████████████████████████| 51kB 3.2MB/s 
     |████████████████████████████████| 10.1MB 11.6MB/s 
     |████████████████████████████████| 61kB 5.2MB/s 
     |████████████████████████████████| 51kB 3.2MB/s 
     |████████████████████████████████| 8.3MB 7.6MB/s 
     |████████████████████████████████| 747kB 43.9MB/s 
     |████████████████████████████████| 61kB 6.6MB/s 
     |████████████████████████████████| 378kB 36.0MB/s 
     |█

In [ ]:
#########################################  DATA REQUESTS #############################################

# To provide current date and time in an format understandable by our API
from datetime import datetime 
from datetime import timedelta
# Properly display html docs before we could work on it
from lxml import html 
# Requests HTML data from websites and api 
import requests 
from requests import Request, Session

# Connecting with an API without any exceptions
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects 

# Clean the result from API's
import json  
from bs4 import BeautifulSoup

##########################################  DATA MINING ##############################################

# To get posts from reddit
import praw

# To get the current stock prices
import yfinance as yf  
from yahoo_fin.stock_info import get_data

# Connect to twitter API
from tweepy import OAuthHandler
from tweepy.streaming import StreamListener
import tweepy

from urllib.request import urlopen

# Library to set up email alerts 
import smtplib

#######################################  DATA MANIPULATION ###########################################

import numpy as np
import pandas as pd
import csv
import re
from textblob import TextBlob
import string
import os
import time

#######################################  DATA VISUALIZATION ###########################################
 
import plotly.express as px 
import plotly.graph_objects as go
import matplotlib.pylab as plt
%matplotlib inline
import seaborn as sns
import plotly.figure_factory as ff
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import mpld3
import plotly

########################################### DATA TRANSFER ############################################

from fastapi import FastAPI
from colabcode import ColabCode 

#######################################################################################################

In [ ]:
# To get the number of posts of a stock subreddit we choose 
def Analyze_Reddit_Posts(stocks,df, common_words):
    """
    Count the total occurences and upvotes of every stock on reddit by 
    crossreferencing it with the DataFrame of reddit post we retreived earlier.

    Parameters
    ----------
    stocks       : pd.DataFrame
    df           : pd.DataFrame
    common_words : list

    Returns
    -------
    pd.DataFrame
    """
    ############################ Analyze Reddit Posts ##########################

    result = [] 
    
    for stock in stocks.itertuples():
      sum ,count = 0, 0
      stock_ = stock.Stock_Name
      for item in df.itertuples():

        # If the stock name occurred in either Title or Content of the post 
        if stock_ in item.Title or stock_ in item.Content:
          sum = sum + item.Upvotes
          count = count + 1
    
      # Append data if the stock:
      #                           Occurred More than once
      #                           Not a common word 
      #                           The name is of len > 4

      if count > 0 and len(stock.Stock_Name)>4 and stock_ not in common_words:
        result.append({'Stock_Name' : stock.Stock_Name,
                       'Ticker' : stock.Ticker,
                       'Number_Of_Occurences' : count,
                       'Total_Upvotes' : sum})
    
    ############################################################################

    return pd.DataFrame(result)

In [ ]:
# To get the number of posts of a stock subreddit we choose 
def Analyze_Reddit_Crypto(cryptos,df, common_words):
    """
    Count the total occurences and upvotes of every crypto on reddit by 
    crossreferencing it with the DataFrame of reddit post we retreived earlier.

    Parameters
    ----------
    cryptos      : pd.DataFrame
    df           : pd.DataFrame
    common_words : list

    Returns
    -------
    pd.DataFrame
    """
    ############################ Analyze Reddit Posts ##########################

    result = [] 
    
    for crypto in cryptos.itertuples():
      sum ,count = 0, 0
      crypto_ = crypto.Name
      for item in df.itertuples():

        # If the stock name occurred in either Title or Content of the post 
        if crypto_ in item.Title or crypto_ in item.Content:
          sum = sum + item.Upvotes
          count = count + 1
    
      # Append data if the stock:
      #                           Occurred More than once
      #                           Not a common word 
      #                           The name is of len > 4

      if count > 0 and len(crypto_)>4 and crypto_ not in common_words:
        result.append({'Name' : crypto_,
                       'Symbol' : crypto.Symbol,
                       'Number_Of_Occurences' : count,
                       'Total_Upvotes' : sum})
    
    ############################################################################

    return pd.DataFrame(result)

In [ ]:
def Analyze_tweets(stocks, df, common_words):
    """
    Count the total occurences and retweets of every stock on twitter by 
    crossreferencing it with the DataFrame of tweets we retreived earlier.

    Parameters
    ----------
    stocks       : pd.DataFrame
    df           : pd.DataFrame
    common_words : list

    Returns
    -------
    pd.DataFrame
    """
    ############################### Analyze Tweets #############################
    
    result = []
    for stock in stocks.iterrows():
      sum, count = 0, 0
      stock_ = stock[1]['Stock_Name']
      for item in df.itertuples():
        if stock_ in item.text:
          sum = sum + item.retweet_count
          count = count + 1
      
      # Append data if the stock:
      #                           Occurred More than once
      #                           Not a common word 
      #                           The name is of len > 4

      if count > 0 and len(stock_) > 4 and stock_ not in common_words:
        result.append({'Stock_Name' : stock_, 
                       'Ticker': stock[1]['Ticker'], 
                       'Number_Of_Occurences' : count, 
                       'Total_retweets' : sum})
        
    ############################################################################
    return pd.DataFrame(result)

In [ ]:
# To get the number of posts of a stock subreddit we choose 
def Analyze_Tweets_Crypto(cryptos,df, common_words):
    """
    Count the total occurences and upvotes of every crypto on reddit by 
    crossreferencing it with the DataFrame of reddit post we retreived earlier.

    Parameters
    ----------
    cryptos      : pd.DataFrame
    df           : pd.DataFrame
    common_words : list

    Returns
    -------
    pd.DataFrame
    """
    ############################ Analyze Reddit Posts ##########################

    result = [] 
    
    for crypto in cryptos.itertuples():
      sum ,count = 0, 0
      crypto_ = crypto.Name
      for item in df.itertuples():

        # If the stock name occurred in either Title or Content of the post 
        if crypto_ in item.text:
          sum = sum + item.retweet_count
          count = count + 1
    
      # Append data if the stock:
      #                           Occurred More than once
      #                           Not a common word 
      #                           The name is of len > 4

      if count > 0 and len(crypto_)>4 and crypto_ not in common_words:
        result.append({'Name' : crypto_,
                       'Symbol' : crypto.Symbol,
                       'Number_Of_Occurences' : count,
                       'Total_Retweets' : sum})
    
    ############################################################################

    return pd.DataFrame(result)